## APIS

# TextBlob
https://textblob.readthedocs.io/en/stable/

Prise en charge de la langue française pour TextBlob :
https://pypi.org/project/textblob-fr/

# MyMemory translated
(API REST)

https://mymemory.translated.net/doc/spec.php

## Imports

In [1]:
"""
json aide à transformer les données entre un format Python et un format que les serveurs web comprennent;
c'est pour l'échange de données entre un serveur et une application web.
"""

import json 


import requests

"""
requests est une bibliothèque permettant à Python d'envoyer des requêtes HTTP et 
d'interagir avec des APIs ou des sites web en envoyant des demandes et 
en récupérant des réponses.
"""

from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer 

# PatternTagger = Part-of-speech tagging
# PatternAnalyzer = Sentiment analysis

## Analyse des sentimens sur les textes sources + traduction
1) Utiliser TextBlob pour effectuer une analyse des sentiments sur deux paragraphes distincts, tirés d'un article contenu dans CAMille. L'article en question étant le fichier intitulé "KB_JB92_1860-02-09_01-00003.pdf" ;
2) Utiliser l'API REST de MyMemory translated pour traduire les deux paragraphes.

In [2]:
## ETAPE 1  > effectuer l'analyse des sentiments sur les textes sources

# Initialisation de Blobber avec PatternTagger et PatternAnalyzer pour le français
tb_fr = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

# Liste des textes à analyser
texts = [
    "ANGLETERRE. Voici en quels termes le Morning-Post annonce l'ajournement de l'exposé financier de M. Gladstone : Nous sommes priés d'annoncer que le chancelier de l'échiquier a été alité depuis le conseil de cabinet de vendredi. Dans ces circonstances, son médecin a cru devoir lui conseiller de ne pas s'occuper des affaires pendant un jour ou deux ; mais jeudi ou vendredi, M. Gladstone espère pouvoir présenter son exposé financier à la Chambre des communes.",
    "L'honorable M. de Boe vous a parlé de ce qui se fait en France, de l'excellence des cours, qui sont très-suivis et qui ne sont pas sujets à examen. Je ne crains pas de dire que dans quelques années les cours à certificats seront les mieux donnés el auront le plus d'attrait. De plus, l'épreuve est trop courte encore pour qu'on puisse juger des résultats. S'il fallait les apprécier dès à présent, je prouverais que le système a déjà produit une grande partie des résultats qu'on en attendait."
]

# URL de l'API REST pour la traduction
api_url = "https://api.mymemory.translated.net/get"

# Liste pour stocker les traductions à réaliser
traductions = []

# Effectuer l'analyse des sentiments pour chaque texte (en français)
for text in texts:
    blob_fr = tb_fr(text)
    sentiment_polarity_fr = blob_fr.sentiment[0]
    sentiment_subjectivity_fr = blob_fr.sentiment[1]
    print(f"Texte source: {text}")
    print(f"Sentiment | Polarité : {sentiment_polarity_fr}, Subjectivité : {sentiment_subjectivity_fr}")
    print
    
    """
    sentiment : Renvoie un tuple de la forme (polarity, subjectivity) où polarity est un float dans la plage [-1.0, 1.0] 
    et subjectivity est un float dans la plage [0.0, 1.0], où 0.0 est très objectif et 1.0 est très subjectif. 
    Le type de retour est un namedtuple de la forme Sentiment(polarity, subjectivity).
    (https://textblob.readthedocs.io/en/stable/api_reference.html#textblob.blob.TextBlob.sentiment)
    """


## ETAPE 2 > traduire les textes en anglais

# Configurer les paramètres de la requête
    params = {
        "q": text, # texte à traduire
        "langpair": "fr|en" # combinaison linguistique
    }
    
    # Envoyer la requête
    response = requests.get(api_url, params=params)
    if response.status_code == 200: # vérifier si la requête a réussi
        traduction = response.json()["responseData"]["translatedText"] 
        traductions.append(traduction) # ajouter la traduction à la liste
    else:
        print(f"Erreur {response.status_code}: Impossible de traduire le texte.")
    print()

Texte source: ANGLETERRE. Voici en quels termes le Morning-Post annonce l'ajournement de l'exposé financier de M. Gladstone : Nous sommes priés d'annoncer que le chancelier de l'échiquier a été alité depuis le conseil de cabinet de vendredi. Dans ces circonstances, son médecin a cru devoir lui conseiller de ne pas s'occuper des affaires pendant un jour ou deux ; mais jeudi ou vendredi, M. Gladstone espère pouvoir présenter son exposé financier à la Chambre des communes.
Sentiment | Polarité : -0.10999999999999999, Subjectivité : 0.375

Texte source: L'honorable M. de Boe vous a parlé de ce qui se fait en France, de l'excellence des cours, qui sont très-suivis et qui ne sont pas sujets à examen. Je ne crains pas de dire que dans quelques années les cours à certificats seront les mieux donnés el auront le plus d'attrait. De plus, l'épreuve est trop courte encore pour qu'on puisse juger des résultats. S'il fallait les apprécier dès à présent, je prouverais que le système a déjà produit un

In [3]:
# Enregistrer les traductions dans un fichier .json (afin que les données restent accessibles pour l'autre cellule)
with open('traductions.json', 'w') as f:
    json.dump(traductions, f)

## Analyse des sentiments sur les traductions

In [5]:
## ETAPE 3  > effectuer l'analyse des sentiments sur les textes cibles

# Charger les traductions depuis le fichier .json
with open('traductions.json', 'r') as f:
    traductions = json.load(f)

# Initialisation de Blobber pour l'analyse en anglais
tb_en = Blobber()

# Étape 3: Analyse des sentiments en anglais
for traduction in traductions:
    blob_en = tb_en(traduction)
    sentiment_polarity_en = blob_en.sentiment[0]
    sentiment_subjectivity_en = blob_en.sentiment[1]
    print(f"Texte traduit: {traduction}")
    print(f"Sentiment | Polarity: {sentiment_polarity_en}, Subjectivity: {sentiment_subjectivity_en}")
    print

Texte traduit: ENGLAND. Here is how the Morning Post announces the adjournment of Mr. Gladstone's financial statement: We are asked to announce that the Chancellor of the Exchequer has been bedridden since Friday's cabinet meeting. Under these circumstances, his doctor felt it necessary to advise him not to take care of business for a day or two; but on Thursday or Friday, Mr. Gladstone hopes to be able to present his financial statement to the House of Commons.
Sentiment | Polarity: 0.1, Subjectivity: 0.325
Texte traduit: The Honourable Mr de Boe has spoken to you about what is being done in France, about the excellence of the courses, which are very well attended and which are not subject to examination. I am not afraid to say that in a few years the certificate courses will be the best given and will have the most appeal. In addition, the test is still too short for us to judge the results. If we were to appreciate them now, I would prove that the system has already produced much of